# Decepticons: Corrupted Transformers Breach Privacy in Federated Learning for Language Models

This notebook shows an example for the threat model and attack described in "Decepticons: Corrupted Transformers Breach Privacy in Federated Learning for Language Models
". This example deviates from the other "honest-but-curious" server models and investigates a malicious server that may send malicious server updates. The attack succeeds for a range of common transformer architectures and works merely by sending a single malicious query to the user model.

In this notebook, we attack the tiny transformer model discussed in "Advances and Open Problems in Federated Learning" (https://arxiv.org/abs/1912.04977). The model architecture is unchanged.



Paper URL: https://arxiv.org/abs/2201.12675

### Abstract:
A central tenet of Federated learning (FL), which trains models without centralizing user data, is privacy. However, previous work has shown that the gradient updates used in FL can leak user information. While the most industrial uses of FL are for text applications (e.g. keystroke prediction), nearly all attacks on FL privacy have focused on simple image classifiers. We propose a novel attack that reveals private user text by deploying malicious parameter vectors, and which succeeds even with mini-batches, multiple users, and long sequences. Unlike previous attacks on FL, the attack exploits characteristics of both the Transformer architecture and the token embedding, separately extracting tokens and positional embeddings to retrieve high-fidelity text. This work suggests that FL on text, which has historically been resistant to privacy attacks, is far more vulnerable than previously thought.

### Startup

In [1]:
try:
    import breaching
except ModuleNotFoundError:
    # You only really need this safety net if you want to run these notebooks directly in the examples directory
    # Don't worry about this if you installed the package or moved the notebook to the main directory.
    import os; os.chdir("..")
    import breaching
    
import torch
%load_ext autoreload
%autoreload 2

# Redirects logs directly into the jupyter notebook
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

### Initialize cfg object and system setup:

This will load the full configuration object. This includes the configuration for the use case and threat model as `cfg.case` and the hyperparameters and implementation of the attack as `cfg.attack`. All parameters can be modified below, or overriden with `overrides=` as if they were cmd-line arguments.

In [2]:
cfg = breaching.get_config(overrides=["attack=decepticon", "case=10_causal_lang_training", 
                                     "case/server=malicious-transformer", "case/data=wikitext"])
          
device = torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=getattr(torch, cfg.case.impl.dtype))
setup

Investigating use case causal_lang_training with server type malicious_transformer_parameters.


{'device': device(type='cpu'), 'dtype': torch.float32}

### Modify config options here

You can use `.attribute` access to modify any of these configurations for the attack, or the case:

In [3]:
cfg.case.user.num_data_points = 8 # How many sentences?
cfg.case.user.user_idx = 1 # From which user?
cfg.case.data.shape = [32] # This is the sequence length

cfg.case.server.has_external_data = False  # ext data is not strictly necessary
# cfg.case.data.tokenizer = "word-level"

cfg.attack.matcher = "corrcoef"
# Attack hyperparameters:

# this option requires installation of `k-means-constrained` which can be tricky:
# If this doesn't work for you, falling back to "dynamic-threshold" is still a decent option.
cfg.attack.sentence_algorithm = "k-means" 

### Instantiate all parties

The following lines generate "server, "user" and "attacker" objects and print an overview of their configurations.

In [4]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Model architecture transformer3 loaded with 10,800,433 parameters and 0 buffers.
Overall this is a data ratio of   42189:1 for target shape [8, 32] given that num_queries=1.
User (of type UserSingleStep) with settings:
    Number of data points: 8

    Threat model:
    User provides labels: False
    User provides buffers: False
    User provides number of data points: True

    Data:
    Dataset: wikitext
    user: 1
    
        
Server (of type MaliciousTransformerServer) with settings:
    Threat model: Malicious (Parameters)
    Number of planned queries: 1
    Has external/public data: False

    Model:
        model specification: transformer3
        model state: default
        

    Secrets: {}
    
Attacker (of type DecepticonAttacker).


### Simulate an attacked FL protocol

This exchange is a simulation of a single query in a federated learning protocol. The server sends out a `server_payload` and the user computes an update based on their private local data. This user update is `shared_data` and contains, for example, the parameter gradient of the model in the simplest case. `true_user_data` is also returned by `.compute_local_updates`, but of course not forwarded to the server or attacker and only used for (our) analysis.

In [5]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)  

Found attention of shape torch.Size([288, 96]).
Computing feature distribution before the probe layer Linear(in_features=96, out_features=1536, bias=True) from random tokens.
Feature mean is -0.34829866886138916, feature std is 1.1387989521026611.
Computing user update on user 1 in model mode: eval.


In [6]:
user.print(true_user_data)

 The Tower Building of the Little Rock Arsenal, also known as U.S. Arsenal Building, is a building located in MacArthur Park in downtown Little Rock, Arkansas
. Built in 1840, it was part of Little Rock's first military installation. Since its decommissioning, The Tower Building has housed two museums. It
 was home to the Arkansas Museum of Natural History and Antiquities from 1942 to 1997 and the MacArthur Museum of Arkansas Military History since 2001. It has also been the
 headquarters of the Little Rock Æsthetic Club since 1894. 
 The building receives its name from its distinct octagonal tower. Besides being the last
 remaining structure of the original Little Rock Arsenal and one of the oldest buildings in central Arkansas, it was also the birthplace of General Douglas MacArthur, who became the supreme
 commander of US forces in the South Pacific during World War II. It was also the starting place of the Camden Expedition. In 2011 it was named as one of
 the top 10 attractions in

### Reconstruct user data:

Now we launch the attack, reconstructing user data based on only the `server_payload` and the `shared_data`. 

For this attack, we also share secret information from the malicious server with the attack (`server.secrets`), which here is the location and structure of the imprint block.

In [7]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], server.secrets, 
                                                      dryrun=cfg.dryrun)

Recovered tokens tensor([[   12,    13,    29,    31,    50,    82,   198,   220,   228,   257,
           262,   278,   284,   286,   287,   290,   301,   318,   319,   340,
           355,   373,   379,   383,   416,   422,   468,   471,   508,   511,
           530,   547],
        [  554,   587,   632,   635,   636,   663,   705,   717,   734,   764,
           832,   835,   837,   838,   852,   867,   871,   938,   959,  1141,
          1181,  1201,  1204,  1248,  1279,  1294,  1295,  1353,  1363,  1438,
          1810,  1900],
        [ 2159,  2422,  2488,  2520,  2581,  2615,  2627,  2656,  2813,  2873,
          2882,  2954,  3250,  3386,  3399,  3411,  3599,  3611,  3700,  3706,
          4318,  4619,  4631,  4645,  4920,  5140,  5824,  5878,  6184,  6289,
          6427,  6587],
        [ 6832,  7310,  7443,  7703,  8211,  8308,  8309,  8765,  9436,  9594,
          9988, 10251, 10433, 10580, 10807, 11583, 11819, 12006, 12068, 12547,
         12842, 13325, 13837, 14538, 15796

Next we'll evaluate metrics, comparing the `reconstructed_user_data` to the `true_user_data`.

In [8]:
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, order_batch=True, compute_full_iip=False, 
                                    cfg_case=cfg.case, setup=setup)

Starting evaluations for attack effectiveness report...
Using default tokenizer.
METRICS: | Accuracy: 0.8867 | S-BLEU: 0.79 | FMSE: 1.1854e-03 | 
 G-BLEU: 0.76 | ROUGE1: 0.89| ROUGE2: 0.79 | ROUGE-L: 0.88| Token Acc T:96.48%/A:98.82% | Label Acc: 96.48%


Finally, we also plot the reconstructed data (marking attack confidence):

In [9]:
user.print_with_confidence(reconstructed_user_data)

 The  Tower  Building  of  the  in  Rock  Arsenal ,  also  known  as  U . S .  Arsenal  Building ,  is  a  building  located  in  MacArthur  Park  in  downtown  Little  Rock ,  Dupl 

.  Built  in  1840 ,  in  was  part  of  Little  Rock  ' s  first  military  installation .  Since  its  decom mission ing ,  The  Tower  Building  has  housed  two  museums . asse 

 Arkansas  home  to  it  Arkansas  Museum  of  Natural  History  and  Antiqu ities  from  1942  to  1997  and  the  MacArthur  Oklahoma  of  Arkansas  Military  History  since  2001 .  It  has  also  been  last 

 It  of  the  Little  Rock  � � st hetic  the  since  18 94 .   
  supreme  building  receives  its  name  from  its  distinct  oct agonal  tower  Museum  Besides  being  headquarters  bumps 

 remaining  structure  of  the  original  Little  Rock  Arsenal  and  one  of  the  oldest  buildings  in  central  Arkansas ,  it  was  also  the  birthplace  of  General  the  MacArthur ,  who  ebook  the 437 

 commander  of

..and compare to the ground truth text:

In [10]:
user.print_and_mark_correct(reconstructed_user_data, true_user_data)

 The  Tower  Building  of  the  in  Rock  Arsenal ,  also  known  as  U . S .  Arsenal  Building ,  is  a  building  located  in  MacArthur  Park  in  downtown  Little  Rock ,  Dupl 

.  Built  in  1840 ,  in  was  part  of  Little  Rock  ' s  first  military  installation .  Since  its  decom mission ing ,  The  Tower  Building  has  housed  two  museums . asse 

 Arkansas  home  to  it  Arkansas  Museum  of  Natural  History  and  Antiqu ities  from  1942  to  1997  and  the  MacArthur  Oklahoma  of  Arkansas  Military  History  since  2001 .  It  has  also  been  last 

 It  of  the  Little  Rock  � � st hetic  the  since  18 94 .   
  supreme  building  receives  its  name  from  its  distinct  oct agonal  tower  Museum  Besides  being  headquarters  bumps 

 remaining  structure  of  the  original  Little  Rock  Arsenal  and  one  of  the  oldest  buildings  in  central  Arkansas ,  it  was  also  the  birthplace  of  General  the  MacArthur ,  who  ebook  the 437 

 commander  of

### Notes:
* There are a variety of hyperparameters to the attack which are set to reasonable defaults. Performance of the attack could be improved in some unusual use cases (datasets or models) by tuning these parameters further.
* In this example, dropout is disabled under the assumption that this is a parameter that can be controlled in the server update. The optimal attack simply disables dropout. However, the attack can still succeed when dropout is enforced by the user, albeit with a minor loss in reconstruction quality.
* Try increasing `num_data_points` or `data.shape`